In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
import warnings 
warnings.filterwarnings('ignore')

In [86]:
from ast import literal_eval

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer

In [88]:
path_to_dataset = r'C:\Users\roszs\1PyFolder\the-movies-dataset\movies_metadata_fixed.csv'
dataset = pd.read_csv(path_to_dataset)

In [89]:
literal_eval(dataset['genres'].iloc[0])
[i['name'] for i in literal_eval(dataset['genres'].iloc[0])]
dataset['genres'] = dataset['genres'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [90]:
dataset['year'] = pd.to_datetime(dataset['release_date']).dt.year

In [91]:
credits = pd.read_csv(r'C:\Users\roszs\1PyFolder\the-movies-dataset\credits.csv')
keywords = pd.read_csv(r'C:\Users\roszs\1PyFolder\the-movies-dataset\keywords.csv')

In [92]:
dataset = pd.merge(dataset, credits, on='id')
dataset = pd.merge(dataset, keywords, on='id')

In [93]:
links_small = pd.read_csv(r'C:\Users\roszs\1PyFolder\the-movies-dataset\links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [94]:
smd = dataset[dataset['id'].isin(links_small)]
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415,1995.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413,1995.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92,1995.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34,1995.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173,1995.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [95]:
literal_eval(smd['cast'].iloc[0])
[i['name'] for i in literal_eval(smd['cast'].iloc[0])]
smd['cast'] = smd['cast'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [96]:
smd['cast'] = smd['cast'].apply(lambda x: x[:5] if len(x) >= 5 else x)
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['cast_str'] = smd['cast'].apply(lambda x: ' '.join(x))
smd['cast_str'].head()

0    tomhanks timallen donrickles jimvarney wallace...
1    robinwilliams jonathanhyde kirstendunst bradle...
2    waltermatthau jacklemmon ann-margret sophialor...
3    whitneyhouston angelabassett lorettadevine lel...
4    stevemartin dianekeaton martinshort kimberlywi...
Name: cast_str, dtype: object

In [97]:
smd['crew'] = smd['crew'].apply(literal_eval)
smd['crew'].iloc[0][0]
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
smd['director'] = smd['crew'].apply(get_director)
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd.director.head()

0      johnlasseter
1       joejohnston
2      howarddeutch
3    forestwhitaker
4      charlesshyer
Name: director, dtype: object

In [98]:
smd['keywords'].head().apply(literal_eval).iloc[0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [99]:
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [100]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1)\
       .stack()\
       .reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s = s[s > 1]

In [101]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words
smd['keywords'] = smd['keywords'].apply(filter_keywords)

In [102]:
def stem_keywords(x):
    stemmed_tokens = []
    for token in x:
        try:
            new_token = stemmer.stem(token)
            stemmed_tokens.append(new_token)
        except:
            stemmed_tokens.append(token)
    return stemmed_tokens

smd['keywords'] = smd['keywords'].apply(lambda x: stem_keywords(x))
smd['keywords'] = smd['keywords'].apply(lambda x: [i.replace(" ", "").lower() for i in x])

In [103]:
smd['keywords_str'] = smd['keywords'].apply(lambda x: ' '.join([str(i) for i in x]))

In [104]:
def concat_fields(data):
    concat = data['keywords'] + data['cast'] + [data['director']] + data['genres']
    result = ' '.join([str(i).lower() for i in concat])
    return result
smd['soup'] = smd.apply(lambda x: concat_fields(x), axis=1)

In [105]:
smd['title'].iloc[0], smd['soup'].iloc[0]

('Toy Story',
 'jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife tomhanks timallen donrickles jimvarney wallaceshawn johnlasseter animation comedy family')

In [106]:
count = CountVectorizer(ngram_range = (1,2), min_df = 2)
count_matrix = count.fit_transform(smd['soup'])
count_matrix.shape

(9219, 21542)

In [107]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim.shape

(9219, 9219)

In [108]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [109]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [122]:
get_recommendations('Euphoria').head(10)

KeyError: 'Euphoria'